### Calculating the ITF heat flux transport using WMT diagnostics

In [16]:
import xarray as xr
import numpy as np
base = '/g/data/e14/mv7494/access-om2/archive/'
from dask.distributed import Client
client = Client(n_workers=4)         # careful: do not run with 16 workers here as it will crash
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35757 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:39777 Dashboard: /proxy/35757/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [17]:
%%time
average_DT = xr.open_mfdataset(base + '1deg_jra55_iaf_branch2000_tropics_40SN/' + 'output*/ocean/ocean_wmass.nc').average_DT
average_DT = np.mean(average_DT.values.astype('timedelta64[s]')).astype(int) # convert to seconds
# [seconds, seconds2]

round_accuracy = 1

for transect in [0,1]:
    if transect == 0:
        print('Transect VIa: Indonesian Throughflow Horizontal')
        lat_cell = 127; lon_cell = [24,54] #
    if transect == 1:
        print('Transect VIb: Indonesian Throughflow Vertical')
        lat_cell = [87,119]; lon_cell = [62] #        
        
    archive = '1deg_jra55_iaf_branch2000_extratropics_40SN/' # use the correct time integration here for the output in monthly resolution

    # open wmass.nc file
    if transect == 0:
        def sum_neutral(ds):
            dsnew = ds.isel(grid_yu_ocean=lat_cell, grid_xt_ocean=slice(lon_cell[0],lon_cell[1])).sum(['neutral']) # grid_yu_ocean=71 -> 35°S, =63 -> 42.5°S
            return dsnew
    else:
        def sum_neutral(ds):
            dsnew = ds.isel(grid_yt_ocean=slice(lat_cell[0],lat_cell[1]), grid_xu_ocean=lon_cell).sum(['neutral'])
            return dsnew
            
    wmass = xr.open_mfdataset(base + archive + 'output*/ocean/ocean_wmass.nc', parallel=True, preprocess = sum_neutral)

    if transect == 0: # if not vertical ITF transport
        # add together the different diagnostics from the advection, submeso, gm and diffusive mixing schemes
        # these fluxes are cp*diagnostic*dxt*rho_dzt*temp binned to neutral density (Watt)
        temp_flux_on_nrho = (wmass.temp_yflux_adv_on_nrho + wmass.temp_yflux_submeso_on_nrho + wmass.temp_yflux_gm_on_nrho + wmass.temp_yflux_ndiffuse_on_nrho)
        temp_flux_on_nrho = (temp_flux_on_nrho*average_DT).cumsum(['time']).sel(time=slice('1972','2017'))
        temp_flux_on_nrho = temp_flux_on_nrho.groupby('time.year').mean('time').sum(['grid_xt_ocean']).load() # take annual mean and sum over the full transect length and load data

    if transect == 1:
        temp_flux_on_nrho = (wmass.temp_xflux_adv_on_nrho + wmass.temp_xflux_submeso_on_nrho + wmass.temp_xflux_gm_on_nrho + wmass.temp_xflux_ndiffuse_on_nrho)
        temp_flux_on_nrho = (temp_flux_on_nrho*average_DT).cumsum(['time']).sel(time=slice('1972','2017'))
        temp_flux_on_nrho = temp_flux_on_nrho.groupby('time.year').mean('time').sum(['grid_yt_ocean']).load()

    if transect == 0:
        ITF_horizontal = temp_flux_on_nrho
    else:
        ITF_vertical = temp_flux_on_nrho

    print('iteration done')
    print('--------------')
        
        

Transect VIa: Indonesian Throughflow Horizontal
iteration done
--------------
Transect VIb: Indonesian Throughflow Vertical


distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distribu

KilledWorker: ("('open_dataset-slice-getitem-862a85fcc7112a6a16a0259b1998ea3a', 0, 0, 0, 0)", <Worker 'tcp://127.0.0.1:46437', name: 3, memory: 0, processing: 4>)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
